# 1. 키워드 추출

## 1-1 제목에서 추출

In [1]:
# 불필요한 경고 출력을 방지
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split


In [3]:
# '-' 깨짐 해결 모듈
import matplotlib.pylab as plt

import matplotlib as mpl

plt.rcParams["font.family"] = "Malgun Gothic"  # For Windows
plt.rcParams["font.size"] = 12
plt.rcParams["figure.figsize"] = (8,4)
print(plt.rcParams["font.family"])

# 마이너스 깨짐 해결
mpl.rcParams["axes.unicode_minus"] = False

['Malgun Gothic']


### 데이터 읽기

In [4]:
# techworld
df_bloter = pd.read_csv('../data_crawling_D/before_keywords/news_bloter_t_202306231600.csv')
print(df_bloter.shape)

(193, 8)


In [5]:
df_bloter.head()


,news_date,news_title,news_text_sm,url_in,news_writer,tags_string,thumb_addr,news_site
0,2023.05.25 19:02,"지그재그 팝업 스토어, '백문이 불여일견' [현장+]",기자는 끊을 수 없을 만큼 인터넷 쇼핑에 길들여졌다. 하지만 취재나 여가를 즐기며 ...,https://www.bloter.net/news/articleView.html?i...,안신혜 기자,"#카카오스타일,#지그재그,#온라인플랫폼,#패션플랫폼,#더현대서울,#팝업스토어",https://cdn.bloter.net/news/thumbnail/202305/6...,Bloter
1,2023.05.25 19:02,"지그재그 팝업 스토어, '백문이 불여일견' [현장+]",기자는 끊을 수 없을 만큼 인터넷 쇼핑에 길들여졌다. 하지만 취재나 여가를 즐기며 ...,https://www.bloter.net/news/articleView.html?i...,안신혜 기자,"#카카오스타일,#지그재그,#온라인플랫폼,#패션플랫폼,#더현대서울,#팝업스토어",https://cdn.bloter.net/news/thumbnail/202305/6...,Bloter
2,2023.05.24 18:27,지그재그도 앱 밖으로 나왔다…팝업 스토어 운영 차별점은?,카카오스타일의 스타일 커머스 플랫폼 지그재그가 애플리케이션에서 나와 더현대 서울에 ...,https://www.bloter.net/news/articleView.html?i...,안신혜 기자,"#패션플랫폼,#커머스,#지그재그,#카카오스타일,#더현대서울,#팝업스토어",https://cdn.bloter.net/news/thumbnail/202305/6...,Bloter
3,2023.05.24 18:27,지그재그도 앱 밖으로 나왔다…팝업 스토어 운영 차별점은?,카카오스타일의 스타일 커머스 플랫폼 지그재그가 애플리케이션에서 나와 더현대 서울에 ...,https://www.bloter.net/news/articleView.html?i...,안신혜 기자,"#패션플랫폼,#커머스,#지그재그,#카카오스타일,#더현대서울,#팝업스토어",https://cdn.bloter.net/news/thumbnail/202305/6...,Bloter
4,2023.05.23 18:25,"네이버·카카오 '뉴스제평위 중단'에 뿔난 인터넷신문…""심사 재개 일정 공개하라""",네이버와 카카오가 최근 뉴스제휴 심사를 무기한 중단한 가운데 인터넷신문들이 뉴스제휴...,https://www.bloter.net/news/articleView.html?i...,박현준 기자,"#네이버,#카카오,#뉴스제휴평가위원회",https://cdn.bloter.net/news/thumbnail/202305/6...,Bloter


In [6]:
df_bloter.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   news_date     193 non-null    object
 1   news_title    193 non-null    object
 2   news_text_sm  193 non-null    object
 3   url_in        193 non-null    object
 4   news_writer   193 non-null    object
 5   tags_string   193 non-null    object
 6   thumb_addr    193 non-null    object
 7   news_site     193 non-null    object
dtypes: object(8)
memory usage: 12.2+ KB


In [7]:
# df_techworld.columns
# idx 떼내기
df_bloter = df_bloter[[ 'news_date', 'news_title', 'news_text_sm', 'url_in',
       'news_writer', 'tags_string', 'thumb_addr', 'news_site']]

In [8]:
# df_recentit news_date 수정하기 (맨우측에 . 삭제)
df_bloter['news_date']= df_bloter['news_date'].str.replace(r'\.', '-', regex=True)
df_bloter['news_date']

# [ 출처: https://seong6496.tistory.com/ ]

0      2023-05-25 19:02
1      2023-05-25 19:02
2      2023-05-24 18:27
3      2023-05-24 18:27
4      2023-05-23 18:25
             ...       
188    2022-06-07 10:49
189    2022-06-06 15:00
190    2022-06-06 15:00
191    2022-06-05 09:04
192    2022-06-05 09:04
Name: news_date, Length: 193, dtype: object

In [9]:
# 결측값 채우기
df_bloter.fillna('', inplace=True)
# df_itworld.fillna('')

## KoNLPy로 단어 단위 분해

In [10]:
# datetime(날짜), os 호출

import datetime
import os
# 불용.txt 읽기 위해 csv 파일 호출
import csv
import re
# konlpy 호출
from konlpy.tag import Okt

# 자연어, 불용어 처리 및 wordcloud 생성을 위해
# 집계 메소드
from collections import Counter
from konlpy.tag import Komoran
# https://datascienceschool.net/03%20machine%20learning/03.01.01%20NLTK%20%EC%9E%90%EC%97%B0%EC%96%B4%20%EC%B2%98%EB%A6%AC%20%ED%8C%A8%ED%82%A4%EC%A7%80.html
from nltk.tokenize import word_tokenize


In [11]:
# 모듈화
def get_title_keywords(df):
    # 컬럼 'news_title' 가져오기 
    rdr = df['news_title'] 
    
    # Okt 객체 선언
    okt = Okt()
    ###  불용어 빼기
    # 불용어 파일 읽기
    stop_words = []
    with open('../data_analysis_test/불용.txt', 'r', encoding='utf-8') as file:
       stop_words = file.read().splitlines()
    
    h_word_list = []
    for w_bundle in rdr:
      # print(okt.nouns(w_bundle))
        list = okt.nouns(w_bundle)
        # 특수문자 제거
        tokens = ",".join(list)
        tokens = re.sub('[\{\}\[\]\/?.;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]', '', tokens)
        list = tokens.split(",")
        # 한글자인 명사, ',' 빼기 (예외:웹,앱,폰,챗)
        for i,v in enumerate(list):
            if len(v) < 2 and (v != '웹' and v != '앱' and v != '폰' and v != '챗'):
                list.pop(i)
            elif v == ',':
                list.pop(i)
            # 불용어 제거
            elif i in stop_words:
                list.pop(i)
        list_str = ",".join(list)
       
        h_word_list.append(list_str)
    return h_word_list
    # print(h_word_list)
    # len(techworld_h_word_list)

bloter_h_word_list = get_title_keywords(df_bloter)

## 해당 리스트 목록을 새로운 컬럼으로 추가

In [12]:
df_bloter['news_key'] = bloter_h_word_list
df_bloter['news_key']

0                         지그재그,팝업,스토어,불,일견,현장
1                         지그재그,팝업,스토어,불,일견,현장
2                         지그재그,앱,팝업,스토어,운영,차별
3                         지그재그,앱,팝업,스토어,운영,차별
4      네이버,카카오,뉴스,평위,중단,인터넷,신문,심사,재개,일정,공개,하라
                        ...                  
188                             네이버,유료,구독,서비스
189                      해부,카본,네거티브,네이버,현재,지표
190                      해부,카본,네거티브,네이버,현재,지표
191                     넘버스,마트,고민,거리,물류,센터,마트
192                     넘버스,마트,고민,거리,물류,센터,마트
Name: news_key, Length: 193, dtype: object

## 1-2 news_text_sm에서 키워드 추출하기

In [13]:
# 모듈화
def get_text_keywords(df):
    # 컬럼 'news_title' 가져오기 
    rdr = df['news_text_sm'] 
    
    # Okt 객체 선언
    okt = Okt()
    ###  불용어 빼기
    # 불용어 파일 읽기
    stop_words = []
    with open('../data_analysis_test/불용.txt', 'r', encoding='utf-8') as file:
       stop_words = file.read().splitlines()
    
    h_word_list = []
    for w_bundle in rdr:
      # print(okt.nouns(w_bundle))
        list = okt.nouns(w_bundle)
        # 특수문자 제거
        tokens = ",".join(list)
        tokens = re.sub('[\{\}\[\]\/?.;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]', '', tokens)
        list = tokens.split(",")
        # 한글자인 명사, ',' 빼기 (예외:웹,앱,폰,챗)
        for i,v in enumerate(list):
            if len(v) < 2 and (v != '웹' and v != '앱' and v != '폰' and v != '챗'):
                list.pop(i)
            elif v == ',':
                list.pop(i)
            # 불용어 제거
            elif i in stop_words:
                list.pop(i)
        list_str = ",".join(list)
       
        h_word_list.append(list_str)
    return h_word_list
    # print(h_word_list)
    # len(techworld_h_word_list)

bloter_txt_word_list = get_title_keywords(df_bloter)


In [14]:
df_bloter['news_txt_key'] = bloter_txt_word_list
df_bloter['news_txt_key']

0                         지그재그,팝업,스토어,불,일견,현장
1                         지그재그,팝업,스토어,불,일견,현장
2                         지그재그,앱,팝업,스토어,운영,차별
3                         지그재그,앱,팝업,스토어,운영,차별
4      네이버,카카오,뉴스,평위,중단,인터넷,신문,심사,재개,일정,공개,하라
                        ...                  
188                             네이버,유료,구독,서비스
189                      해부,카본,네거티브,네이버,현재,지표
190                      해부,카본,네거티브,네이버,현재,지표
191                     넘버스,마트,고민,거리,물류,센터,마트
192                     넘버스,마트,고민,거리,물류,센터,마트
Name: news_txt_key, Length: 193, dtype: object

In [15]:
# df_techworld

### 'news_key', 'news_txt_key' 값 합치기

In [25]:
df_bloter['total_keywords'] = df_bloter[['news_key', 'news_txt_key']].apply(','.join, axis=1)
print(df_bloter)

            news_date                                    news_title  \
0    2023-05-25 19:02                 지그재그 팝업 스토어, '백문이 불여일견' [현장+]   
1    2023-05-25 19:02                 지그재그 팝업 스토어, '백문이 불여일견' [현장+]   
2    2023-05-24 18:27               지그재그도 앱 밖으로 나왔다…팝업 스토어 운영 차별점은?   
3    2023-05-24 18:27               지그재그도 앱 밖으로 나왔다…팝업 스토어 운영 차별점은?   
4    2023-05-23 18:25  네이버·카카오 '뉴스제평위 중단'에 뿔난 인터넷신문…"심사 재개 일정 공개하라"   
..                ...                                           ...   
188  2022-06-07 10:49                     네이버는 어떻게 '유료 구독 서비스'를 키웠나   
189  2022-06-06 15:00            [ESG 해부]'카본 네거티브' 바라보는 네이버, 현재 지표는   
190  2022-06-06 15:00            [ESG 해부]'카본 네거티브' 바라보는 네이버, 현재 지표는   
191  2022-06-05 09:04                  [넘버스]B마트 고민거리 '물류센터' 파고든 요마트   
192  2022-06-05 09:04                  [넘버스]B마트 고민거리 '물류센터' 파고든 요마트   

                                          news_text_sm  \
0    기자는 끊을 수 없을 만큼 인터넷 쇼핑에 길들여졌다. 하지만 취재나 여가를 즐기며 ...   
1    기자는 끊을 수 없을 만큼 인터넷 쇼핑에 길들여

### 중복 키워드 처리

In [17]:
# from collections import OrderedDict

# list = '티맥스,티베로,인텔,코리아,비즈니스,연속성,혁신,전략,웨비,개최,티맥스,티베로,인텔,코리아,비즈니스,연속성,혁신,전략,웨비,개최'
# list_sep = list.split(",")
# list_sep

# a = " ".join(OrderedDict.fromkeys(list_sep))
# print(a)




In [ ]:
from collections import OrderedDict

t_keyword_list = []

for keys in df_bloter['total_keywords']:
    # print([keys]) # 일단 전부 ,로 split()해줘야 단어끼리로 분리됨
    keys_list = keys.split(",")
    key_list_str = ','.join(OrderedDict.fromkeys(keys_list))
    # print(key_list_str)
    # 2차원 리스트 만들기
    t_keyword_list.append(key_list_str)

t_keyword_list


In [27]:
df_bloter['total_keywords'] = t_keyword_list
df_bloter['total_keywords'] 

0                         지그재그,팝업,스토어,불,일견,현장
1                         지그재그,팝업,스토어,불,일견,현장
2                         지그재그,앱,팝업,스토어,운영,차별
3                         지그재그,앱,팝업,스토어,운영,차별
4      네이버,카카오,뉴스,평위,중단,인터넷,신문,심사,재개,일정,공개,하라
                        ...                  
188                             네이버,유료,구독,서비스
189                      해부,카본,네거티브,네이버,현재,지표
190                      해부,카본,네거티브,네이버,현재,지표
191                        넘버스,마트,고민,거리,물류,센터
192                        넘버스,마트,고민,거리,물류,센터
Name: total_keywords, Length: 193, dtype: object

### 'news_key', 'news_text_key' 컬럼 삭제

In [28]:
df_bloter.drop(['news_key', 'news_txt_key'], axis=1, inplace=True)

In [20]:
# df_techworld

## df_techworld csv 파일에 반영


In [29]:
df_bloter.to_csv("../data_crawling_D/after_keywords/news_bloter_k_t.csv", mode='w')

# 2. 신뢰도 평가 : news_titlw, news_text_sm의 데이터를 KRWORDRANK로 추출

In [30]:
import pandas as pd
import numpy as np

In [31]:
# 먼저 news_text_sm 컬럼의 값들을 뭉쳐서 하나의 string(각 요소는 '\n'로 나누기)
# 으로 통합
import csv # sql_cwn_data에서 갖고 오기

f = open("../data_crawling_D/after_keywords/news_bloter_k_t.csv", "r", encoding='utf8')
reader = csv.reader(f)
next(reader)
# print(reader)
data = []
for row in reader:
    data.append(row)
    # print(row)
f.close()



In [24]:
# data


In [33]:
# dataframe으로 만들기
df_bloter = pd.DataFrame(data=data, columns=['idx','news_date', 'news_title', 'news_text_sm', 'url_in', 'news_writer', 'tags_string', 'thumb_addr', 'news_site', 'total_keywords'])  
# idx (인덱스) 제거
df_bloter = df_bloter[['news_date', 'news_title', 'news_text_sm', 'url_in', 'news_writer', 'tags_string', 'thumb_addr', 'news_site','total_keywords']]
df_bloter

,news_date,news_title,news_text_sm,url_in,news_writer,tags_string,thumb_addr,news_site,total_keywords
0,2023-05-25 19:02,"지그재그 팝업 스토어, '백문이 불여일견' [현장+]",기자는 끊을 수 없을 만큼 인터넷 쇼핑에 길들여졌다. 하지만 취재나 여가를 즐기며 ...,https://www.bloter.net/news/articleView.html?i...,안신혜 기자,"#카카오스타일,#지그재그,#온라인플랫폼,#패션플랫폼,#더현대서울,#팝업스토어",https://cdn.bloter.net/news/thumbnail/202305/6...,Bloter,"지그재그,팝업,스토어,불,일견,현장"
1,2023-05-25 19:02,"지그재그 팝업 스토어, '백문이 불여일견' [현장+]",기자는 끊을 수 없을 만큼 인터넷 쇼핑에 길들여졌다. 하지만 취재나 여가를 즐기며 ...,https://www.bloter.net/news/articleView.html?i...,안신혜 기자,"#카카오스타일,#지그재그,#온라인플랫폼,#패션플랫폼,#더현대서울,#팝업스토어",https://cdn.bloter.net/news/thumbnail/202305/6...,Bloter,"지그재그,팝업,스토어,불,일견,현장"
2,2023-05-24 18:27,지그재그도 앱 밖으로 나왔다…팝업 스토어 운영 차별점은?,카카오스타일의 스타일 커머스 플랫폼 지그재그가 애플리케이션에서 나와 더현대 서울에 ...,https://www.bloter.net/news/articleView.html?i...,안신혜 기자,"#패션플랫폼,#커머스,#지그재그,#카카오스타일,#더현대서울,#팝업스토어",https://cdn.bloter.net/news/thumbnail/202305/6...,Bloter,"지그재그,앱,팝업,스토어,운영,차별"
3,2023-05-24 18:27,지그재그도 앱 밖으로 나왔다…팝업 스토어 운영 차별점은?,카카오스타일의 스타일 커머스 플랫폼 지그재그가 애플리케이션에서 나와 더현대 서울에 ...,https://www.bloter.net/news/articleView.html?i...,안신혜 기자,"#패션플랫폼,#커머스,#지그재그,#카카오스타일,#더현대서울,#팝업스토어",https://cdn.bloter.net/news/thumbnail/202305/6...,Bloter,"지그재그,앱,팝업,스토어,운영,차별"
4,2023-05-23 18:25,"네이버·카카오 '뉴스제평위 중단'에 뿔난 인터넷신문…""심사 재개 일정 공개하라""",네이버와 카카오가 최근 뉴스제휴 심사를 무기한 중단한 가운데 인터넷신문들이 뉴스제휴...,https://www.bloter.net/news/articleView.html?i...,박현준 기자,"#네이버,#카카오,#뉴스제휴평가위원회",https://cdn.bloter.net/news/thumbnail/202305/6...,Bloter,"네이버,카카오,뉴스,평위,중단,인터넷,신문,심사,재개,일정,공개,하라"
...,...,...,...,...,...,...,...,...,...
188,2022-06-07 10:49,네이버는 어떻게 '유료 구독 서비스'를 키웠나,네이버 유료 구독 회원 서비스 '네이버플러스 멤버십'이 누적 사용자 1000만명 돌...,https://www.bloter.net/news/articleView.html?i...,최창원,"#네이버,#멤버십,#유료,#구독",https://cdn.bloter.net/news/thumbnail/202206/4...,Bloter,"네이버,유료,구독,서비스"
189,2022-06-06 15:00,"[ESG 해부]'카본 네거티브' 바라보는 네이버, 현재 지표는",기업에게 2050년 탄소중립 목표는 단순한 환경보호운동이 아니다. 21세기 기업의 ...,https://www.bloter.net/news/articleView.html?i...,최창원,"#네이버,#ESG,#카본네거티브",https://cdn.bloter.net/news/thumbnail/202206/4...,Bloter,"해부,카본,네거티브,네이버,현재,지표"
190,2022-06-06 15:00,"[ESG 해부]'카본 네거티브' 바라보는 네이버, 현재 지표는",기업에게 2050년 탄소중립 목표는 단순한 환경보호운동이 아니다. 21세기 기업의 ...,https://www.bloter.net/news/articleView.html?i...,최창원,"#네이버,#ESG,#카본네거티브",https://cdn.bloter.net/news/thumbnail/202206/4...,Bloter,"해부,카본,네거티브,네이버,현재,지표"
191,2022-06-05 09:04,[넘버스]B마트 고민거리 '물류센터' 파고든 요마트,3줄 요약· 사회적 거리두기 조치 해제로 배달앱 시장 성장세가 꺾였다는 분석이 나옵...,https://www.bloter.net/news/articleView.html?i...,최창원,"#요기요,#요마트,#B마트,#우아한형제들,#배달의민족",https://cdn.bloter.net/news/thumbnail/202206/4...,Bloter,"넘버스,마트,고민,거리,물류,센터"


## news_title과 news_text_sm를 합친 컬럼 생성하기

In [34]:
df_bloter['title_n_text'] = df_bloter[['news_title', 'news_text_sm']].apply(','.join, axis=1)
# df_techworld['title_n_text']


In [37]:
# 제목과 내용을 합친 str 컬럼을 news_date와 같이 엮어 df화
df_bloter_dnt = df_bloter[['news_date', 'title_n_text']]
# csv 파일로 저장하기
df_bloter_dnt.to_csv("../data_analysis_test/test_data/bloter_dnt.txt", mode='w', header=False, index=False, sep='\t')

In [28]:
# df_cwn_date = df_cwn['news_date'].to_list()
# df_cwn_text = df_cwn['news_text_sm'].to_list()
# df_cwn_date_n_text = {}
# for date, text in zip(df_cwn_date, df_cwn_text):
#     df_cwn_date_n_text[date] = text
                          
# print('dict zip : ', df_cwn_date_n_text)

# df_cwn_text

In [38]:
df_bloter_dnt

,news_date,title_n_text
0,2023-05-25 19:02,"지그재그 팝업 스토어, '백문이 불여일견' [현장+],기자는 끊을 수 없을 만큼 인..."
1,2023-05-25 19:02,"지그재그 팝업 스토어, '백문이 불여일견' [현장+],기자는 끊을 수 없을 만큼 인..."
2,2023-05-24 18:27,"지그재그도 앱 밖으로 나왔다…팝업 스토어 운영 차별점은?,카카오스타일의 스타일 커머..."
3,2023-05-24 18:27,"지그재그도 앱 밖으로 나왔다…팝업 스토어 운영 차별점은?,카카오스타일의 스타일 커머..."
4,2023-05-23 18:25,"네이버·카카오 '뉴스제평위 중단'에 뿔난 인터넷신문…""심사 재개 일정 공개하라"",네..."
...,...,...
188,2022-06-07 10:49,"네이버는 어떻게 '유료 구독 서비스'를 키웠나,네이버 유료 구독 회원 서비스 '네이..."
189,2022-06-06 15:00,"[ESG 해부]'카본 네거티브' 바라보는 네이버, 현재 지표는,기업에게 2050년 ..."
190,2022-06-06 15:00,"[ESG 해부]'카본 네거티브' 바라보는 네이버, 현재 지표는,기업에게 2050년 ..."
191,2022-06-05 09:04,"[넘버스]B마트 고민거리 '물류센터' 파고든 요마트,3줄 요약· 사회적 거리두기 조..."


In [30]:
# f = open("../know_arc_pjt/data_crawling_D/data_sql_total_edited/total_news_t_202306221233.csv", "r", encoding='utf8')
# reader = csv.reader(f)

# for row in reader:
#     data = list(reader)
# # print(data)
# f.close()
# # dataframe으로 만들기
# df_total_t = pd.DataFrame(data=data, columns=['news_date', 'news_title', 'news_text_sm', 'url_in', 'news_writer', 'tags_string', 'thumb_addr', 'news_site'])  
# df_total_t

In [39]:
# 날짜당 제목에서 추출한 키워드 목록
def get_date_text(fname):
    # 튜토리얼에서 이용하는 `fname` 파일은 영화평과 평점이 \t 으로 구분된 two column tsv 파일입니다.
    # 예시는 이 cell 의 output 을 참고하세요.
    with open(fname, encoding='utf-8') as f:
        docs = [doc.lower().replace('\n','').split('\t') for doc in f] 
        docs = [doc for doc in docs if len(doc) == 2]

        if not docs:
            return [], []
        
        dates, texts = zip(*docs)
        return dates, texts

# cwn.kr data (df_cwn_date_n_text)
fname = '../data_analysis_test/test_data/bloter_dnt.txt'
# 이걸로 
dates, texts = get_date_text(fname)
# dates = list(dates)
# texts = list(texts)
# 위쪽의 5개 출력하기
# with open(fname, encoding="utf-8") as f:
#     for _ in range(5):
#         print(next(f).strip())

In [32]:
# dates
# texts

In [40]:
# KR-WordRank에 필요한 라이브러리
import sys
import re
sys.path.append('../')
from krwordrank.word import KRWordRank
from krwordrank.hangle import normalize
import krwordrank
# print(krwordrank.__version__)

단어 추출에 영어/숫자를 포함할 예정이라면 normalize함수를 이용하여 텍스트를 normalize할 것

In [41]:
with open('../data_analysis_test/test_data/bloter_dnt_norm.txt', 'w', encoding='utf-8') as f:
    for date, text in zip(dates, texts):
        text = normalize(text, english=True, number=True)
        # 특수문자는 제거
        text = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", text)
        f.write('%s\t%s\n' % (date, text))

## 해당 데이터에서 핵심 키워드들을 뽑아보자

In [43]:
# df_cwn_date_n_text
fname = '../data_analysis_test/test_data/bloter_dnt_norm.txt'
dates_n, texts_n = get_date_text(fname)
# 여기까진 dates_n, texts_n이 서로 분리되어있음

In [44]:
wordrank_extractor = KRWordRank(
    min_count = 5, # 단어의 최소 출현 빈도수 (그래프 생성 시)
    max_length = 10, # 단어의 최대 길이
    verbose = True
    )

beta = 0.85    # PageRank의 decaying factor beta
max_iter = 10

# 여기서 for문
# text_key_list = []
# for text in texts_n:
keywords, rank, graph = wordrank_extractor.extract(texts_n, beta, max_iter)
    

scan vocabs ... 
num vocabs = 2620
done = 10


In [37]:
# keywords

위와 같이 vocabulary를 미리 설정하거나 decaying factor를 단어별로 다르게 (bias) 할당할 수 있으며, 모든 단어의 랭킹의 총 합은 vocabulary size와 같음. 즉 default decaying factor는 1.0

In [45]:
# 불용어 파일 읽기
stop_words = []
with open('../data_analysis_test/불용.txt', 'r', encoding='utf-8') as file:
   stop_words = file.read().splitlines()
# keyword_list
# # 불용어 제거
pop_list = []
for keyword in keywords:
    # print(keyword)
    # print(type(keyword))
    if keyword in stop_words:
        pop_list.append(keyword)
        # print(keyword)
        
# print(pop_list)
for p in pop_list:
    # print(p)
    keywords.pop(p, None)

# # filtered_tokens =  [token for token in keyword_list[0] if token not in stop_words]
# # filtered_tokens
# keywords

In [46]:
top_keywords = []
# for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:30]:
#     print('%8s:\t%.4f' % (word, r))
top_keywords.append(
    sorted(keywords.items(),
           key=lambda x:x[1],
           reverse=True)[:100]
)


keyword_list = []
for keywords in top_keywords:
    words, ranks = zip(*keywords)
    print(words)
    
    keyword_list.append(words)


('네이버', '카카오', '플랫폼', '서비스', '글로벌', '기업', '검색', '사업', 'ai', '시장', '콘텐츠', '열린', '커머스', '비대면', '뉴스', '이유', '기준', '진료', '메타', '정보', '대표', '계획이다', '자회사', '광고', '모바일', '게임', '사용자', '방안', '운영', '국내', '인수', '3분기', '예정이다', '이달', '온라인', '개발', '배달', '투자', 'gs', '출시', '물류', '보상', 'iec', '국회', '생태계', '리뷰', '기존', '됐다', '자사', '판매', '오픈', '브랜드', '해외', '공개', '사회적', '이날', '추천', 'cic', '회사의', '디지털', '김범수', '3d', '커뮤니티', '서울', '피해', '진행', '배민', '구글', '계열사', '과학기술정보통신부', '민간', '구독', '영업', '장애', 'c2c', '전략', '공시한', '조직을', '미글루', '실적', '자율', '개편', '과기정통부', '구매', '팝업', '상품', '복지부', '쇼핑', '투명성', '전문', '코로나19', 'b2b', '센터', 'pc', '여행', '발표', '요약된다', '폐지', '성장', '발생한')


In [ ]:
for k in range(100):
    
    message = '  --  '.join(
        ['%8s (%.3f)' % (top_keywords[0][k][0],top_keywords[0][k][1])]
         )
    print(message)
# 여기까지가 기사 본문으로부터 키워드 추출

 top 100에서 중복되는 키워드들을 제거하고 차이가 있는 키워드만 추출해서 살펴보겠습니다.

In [69]:
keyword_counter = {}

w_l = []
r_l = []
for keywords in top_keywords:
    words, ranks = zip(*keywords)
    for word in words:
        keyword_counter[word] = keyword_counter.get(word, 0) + 1
        w_l.append(word)
    for rank in ranks:
        r_l.append(rank)

csv_list = list(zip(w_l, r_l))
common_keywords = {word for word, count in keyword_counter.items() if count == 3}
len(common_keywords)

0

In [70]:
# csv_list


In [73]:
df_csv = pd.DataFrame(data=csv_list, columns=['words','ranks'])
df_csv = df_csv.sort_values(by=['ranks'], ascending=False)
df_csv

,words,ranks
0,네이버,16.978843
1,카카오,14.721183
2,플랫폼,10.637819
3,서비스,8.862532
4,글로벌,5.776289
...,...,...
95,발표,1.260716
96,요약된다,1.254400
97,폐지,1.253537
98,성장,1.242535


In [74]:
df_csv.to_csv("../data_crawling_D/keywords/bloter_keywords.csv", mode='w')

## 키워드 분석

## # Reference keywords

In [48]:

# df_techworld
df_techworld_tnt = df_techworld[['news_date', 'total_keywords']]
# df_techworld_tnt

In [ ]:
# 데이터프레임에서 '내용'과 '작성일' 열을 가져옴
import re
from collections import Counter

# 모듈화하기
def get_keyword_monthly_agg(df_techworld_tnt):
    text_data = df_techworld_tnt['total_keywords']
    date_data = df_techworld_tnt['news_date']
    
    
    # 단어 등장 횟수를 저장할 딕셔너리 초기화
    word_counts = {}
    # 월별 단어 집계
    word_n_cnt_list = []
    
    # 데이터프레임을 순회하며 월 별로 단어 등장 횟수 집계
    for text, date in zip(text_data, date_data):
        # 텍스트 토큰화 및 형태소 분석
        
        # 년 추출
        tokens = text.split(",")
        year = date.split('-')[0]
        # 월 추출
        month = date.split('-')[1]
        year_month = year + '-' + month

        # 단어 등장 횟수 집계
        if year_month in word_counts:
            for token in tokens:
                word_counts[year_month][token] = word_counts[year_month].get(token, 0) + 1
        else:
            word_counts[year_month] = {}
            for token in tokens:
                word_counts[year_month][token] = 1
                
    # 횟수가 2이상인 단어만 추출하기
    min_word_count = 2
    top_words_limit = 21 # 최대 20개
    
    # 월 별로 상위 단어 추출
    
    for year_month, counts in word_counts.items():
        # print(f"{month}월")
        # 횟수만 추출하기
        sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
        
        # 횟수가 2이상인 단어 필터링
        filtered_counts = [(word, count) for word, count in sorted_counts if count >= min_word_count]
        
        # 만약 2이상인 단어의 수가 5개 미만이면 모든 단어 추출
        if len(filtered_counts) < top_words_limit:
            filtered_counts = sorted_counts
        # 상위 단어 출력
        top_words = filtered_counts[:top_words_limit]
        for word, count in top_words:
            # print(f" {word} | 등장횟수: {count}")
            word_n_cnt_list.append([year_month, word, count])
        # print(word_n_cnt_list)
        print()
    
    return word_n_cnt_list

tw_key_list = get_keyword_monthly_agg(df_techworld_tnt)
# df_tw_key_h_list

In [55]:
# 리스트를 df으로
df_tw_key_list = pd.DataFrame(tw_key_list, columns=['date','words','cnt'])
print('2차원 리스트로 만든 dataframe:\n', df_tw_key_list)

2차원 리스트로 만든 dataframe:
         date words  cnt
0    2023-05  클라우드   34
1    2023-05    출시   20
2    2023-05    기반   18
3    2023-05   글로벌   18
4    2023-05   솔루션   16
..       ...   ...  ...
247  2022-06    테크   26
248  2022-06   데이터   26
249  2022-06   파트너   24
250  2022-06    지원   22
251  2022-06    지능   22

[252 rows x 3 columns]


In [ ]:
# pivot으로 {인덱스 : date, 컬럼: words, 값: cnt}인 테이블 생성
df_tw_key_pivot = df_tw_key_list.pivot(index='date', columns='words', values='cnt')
df_tw_key_pivot.fillna('0', inplace=True) # 결측값 0처리
df_tw_key_pivot = df_tw_key_pivot.apply(pd.to_numeric)
# df_tw_key_pivot

In [ ]:
df_tw_key_pivot.info()

In [68]:
# 각 단어당 cnt 합산을 구한 뒤 상위 100개 리스트에 담기
df_tw_key_pivot_sum = df_tw_key_pivot.sum()
df_tw_key_pivot_sum_srt = df_tw_key_pivot_sum.sort_values(ascending=False)
df_tw_key_pivot_sum_srt

words
클라우드      452.0
출시        362.0
솔루션       349.0
서비스       300.0
사업        263.0
          ...  
공략          6.0
공공          6.0
이스트소프트      6.0
유공          6.0
티맥스소프트      6.0
Length: 65, dtype: float64

In [69]:
# 인덱스에서 word를 빼내고
df_tw_key_pivot_sum_srt = df_tw_key_pivot_sum_srt.reset_index(drop=False)

In [73]:
ref_key_list = df_tw_key_pivot_sum_srt['words'].to_list()
# ref_list

In [74]:
# 100개 reference용 키워드 추출
top_100_ref_list = ref_key_list[:100]    

In [72]:
top_100_kw_list = []
for k in range(100):
    # print(top_keywords[0][k][0])
    top_100_kw_list.append(top_keywords[0][k][0])
print(top_100_kw_list)

['ai', '클라우드', '기업', '서비스', '솔루션', '데이터', '플랫폼', '디지털', '보안', '개발', '글로벌', '출시', '사업', '국내', '전문', '체결', '시장', '지원', '선정', '프로', '구축', '고객', '관리', '시스템', '업무', '발표', '운영', '한국', 'it', '제품', '산업', '메타', '대표', '서울', '비즈니스', '인공', '진행', '분석', '개최', '소프트', '인증', '파트너', '공급', '스마트', '분야', '연구', '로봇', '혁신', '통합', '스타트업', '협력', '자사', '기능', '투자', '올해', '공개', '교육', '환경', '자동', '획득', '인프라', '정보', '기존', '공공', '콘텐츠', '확대', '강화', '온라인', '오픈', '금융', '코리아', '구글', '확장', '영상', '모델', '3d', '추진', 'kt', '참가', 'mo', 'nft', '차세대', '6월', '성장', '가상', '신규', '인텔', '적용', '오라클', '마이', '가능', '가트너', '버추얼', '애플리케이션', 'digital', '의료', '공식', '매출', 'software', 'aws']


## ROUGE-1,2,l 로 성능 평가

In [76]:
from rouge import Rouge

# 모델로 구한 키워드들
rank_list = top_100_kw_list
rank_str = " ".join(rank_list)
# 실제 헤드라인과 내용에서 집계한 키워드들
ref_str = " ".join(top_100_ref_list)
model_out = [rank_str]
# '다양한', '관련', '것이', '새로운', '최근', '분석', '출처:', '활용', '따라', '한다.', '세계', '공동', '가장', '하는', '현재', '기존', '등을', '점유율은', '모든', '다른', '대해', '많은', '순위', '결과를', '또한', '대비', '10', '신규', '이용', '이를', '주요', '모두', '문제', '그러나', '여러', '출시', '이미', '생성', '가능', '이어', '공개', '발표', '이후', '최신', '전했다', '특히', '이상', '인기', '이는', '하지만', '지수', '이러한', '때문에', '핵심', '6월', '아이', '것은', '그리고', '이에', '같이', '동시에', '각각', '참여', '시간', '전체', '하나', '수준', '물론', '다수', '동안', '9월', '어떤', '전망', '혹은', '양자', '전자', '바로', '여전히'
reference = [ref_str]
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)

# >>>
# {   'rouge-1': {   'f': 0.6279006234427593,
#                    'p': 0.8604497354497355,
#                    'r': 0.5273531655225019},
#     'rouge-2': {   'f': 0.3883256484545606,
#                    'p': 0.5244559362206421,
#                    'r': 0.32954545454545453},
#     'rouge-l': {   'f': 0.6282785202429159,
#                    'p': 0.8122895622895623,
#                    'r': 0.5369305616983636}}

{'rouge-1': {'r': 0.6307692307692307, 'p': 0.41, 'f': 0.4969696921946741},
 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-l': {'r': 0.2153846153846154, 'p': 0.14, 'f': 0.16969696492194689}}